In [1]:
import evaluate
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForTokenClassification,
    TrainingArguments,
    Trainer,
    DataCollatorForTokenClassification,
)

In [2]:
ner_datasets = load_dataset('peoples_daily_ner', cache_dir='./data')
ner_datasets

Using the latest cached version of the module from C:\Users\25338\.cache\huggingface\modules\datasets_modules\datasets\peoples_daily_ner\594461a1b34f61af9346123a420b9ea40f15c0e835562053bf025cef188477f5 (last modified on Mon Jul 31 11:26:45 2023) since it couldn't be found locally at peoples_daily_ner., or remotely on the Hugging Face Hub.


DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 20865
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 2319
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 4637
    })
})

In [3]:
ner_datasets['train'][0]

{'id': '0',
 'tokens': ['海',
  '钓',
  '比',
  '赛',
  '地',
  '点',
  '在',
  '厦',
  '门',
  '与',
  '金',
  '门',
  '之',
  '间',
  '的',
  '海',
  '域',
  '。'],
 'ner_tags': [0, 0, 0, 0, 0, 0, 0, 5, 6, 0, 5, 6, 0, 0, 0, 0, 0, 0]}

In [4]:
ner_datasets['train'].features

{'id': Value(dtype='string', id=None),
 'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'ner_tags': Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'], id=None), length=-1, id=None)}

In [5]:
label_list = ner_datasets['train'].features['ner_tags'].feature.names
label_list

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC']

In [6]:
tokenizer = AutoTokenizer.from_pretrained('./chinese-macbert-base/')

In [7]:
tokenizer(ner_datasets['train'][0]['tokens'], is_split_into_words=True)

{'input_ids': [101, 3862, 7157, 3683, 6612, 1765, 4157, 1762, 1336, 7305, 680, 7032, 7305, 722, 7313, 4638, 3862, 1818, 511, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [8]:
def process_function(examples):
    tokenized_examples = tokenizer(
        examples['tokens'], max_length=128, truncation=True, is_split_into_words=True
    )
    labels = []
    for i, label in enumerate(examples['ner_tags']):
        word_ids = tokenized_examples.word_ids(batch_index=i)
        label_ids = []
        for word_id in word_ids:
            if word_id is None:
                label_ids.append(-100)
            else:
                label_ids.append(label[word_id])
        labels.append(label_ids)
    tokenized_examples['labels'] = labels
    return tokenized_examples

In [9]:
tokenized_datasets = ner_datasets.map(process_function, batched=True)
tokenized_datasets

Map:   0%|          | 0/2319 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 20865
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 2319
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 4637
    })
})

In [15]:
model = AutoModelForTokenClassification.from_pretrained(
    "./chinese-macbert-base/", num_labels=len(label_list)
)

Some weights of the model checkpoint at ./chinese-macbert-base/ were not used when initializing BertForTokenClassification: ['cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized fr

In [11]:
seqeval = evaluate.load('seqeval', cache_dir='./evaluate')
seqeval

EvaluationModule(name: "seqeval", module_type: "metric", features: {'predictions': Sequence(feature=Value(dtype='string', id='label'), length=-1, id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='label'), length=-1, id='sequence')}, usage: """
Produces labelling scores along with its sufficient statistics
from a source against one or more references.

Args:
    predictions: List of List of predicted labels (Estimated targets as returned by a tagger)
    references: List of List of reference labels (Ground truth (correct) target values)
    suffix: True if the IOB prefix is after type, False otherwise. default: False
    scheme: Specify target tagging scheme. Should be one of ["IOB1", "IOB2", "IOE1", "IOE2", "IOBES", "BILOU"].
        default: None
    mode: Whether to count correct entity labels with incorrect I/B tags as true positives or not.
        If you want to only count exact matches, pass mode="strict". default: None.
    sample_weight: Array-like of sha

In [12]:
import numpy as np


def eval_metric(pred):
    predictions, labels = pred
    predictions = np.argmax(predictions, axis=-1)

    ture_predictions = [
        [label_list[p] for p, l in zip(prediction, label) if l != 100]
        for prediction, label in zip(predictions, labels)
    ]

    ture_labels = [
        [label_list[l] for p, l in zip(prediction, label) if l != 100]
        for prediction, label in zip(predictions, labels)
    ]

    result = seqeval.compute(
        predictions=ture_predictions,
        references=ture_labels,
        mode="strict",
        scheme="IOB2",
    )
    return {"f1": result["overall_f1"]}

In [13]:
args = TrainingArguments(
    output_dir="models_for_ner",
    per_device_train_batch_size=64,
    per_device_eval_batch_size=128,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    metric_for_best_model="f1",
    load_best_model_at_end=True,
    logging_steps=50,
)

In [14]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    compute_metrics=eval_metric,
    data_collator=DataCollatorForTokenClassification(tokenizer=tokenizer),
)

In [16]:
trainer.train()

d:\IDE\Anaconda\A\envs\fproject\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/981 [00:00<?, ?it/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
Exception ignored in: <function tqdm.__del__ at 0x000001B24B6B13A0>
Traceback (most recent call last):
  File "d:\IDE\Anaconda\A\envs\fproject\lib\site-packages\tqdm\std.py", line 1145, in __del__
    self.close()
KeyboardInterrupt: 


IndexError: Target 5 is out of bounds.

In [1]:
from transformers import pipeline

In [ ]:
model.config.id2label = dict(enumerate(label_list))

In [ ]:
ner_pipeline = pipeline(
    'token-classification',
    model=model,
    tokenizer=tokenizer,
    device=0,
    aggregation_strategy="simple",
)

In [ ]:
res = ner_pipeline('小明在北京上班')
res

In [ ]:
ner_result = {}
x = '小明在北京上班'
for r in res:
    if r['entity_group'] not in ner_result:
        ner_result[r['entity_group']] = []
    ner_result[r['entity_group']].append(x[r['start'] : r['end']])

ner_result